<div style="max-width:1400px;margin-center: auto">
<img src="images\laptime.png" width="600"/>
</div>

In [1]:
import findspark
findspark.init()
import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local') \
    .appName("laptime") \
    .config("spark.sql.catalogImplementation", "hive") \
    .getOrCreate()

In [2]:
from pyspark.sql.types import StructField, StructType, IntegerType, StringType
from pyspark.sql.functions import col, current_timestamp

In [3]:
lap_time_schema = StructType( fields = [StructField("raceId", IntegerType(), False),
                                        StructField("driverId", IntegerType(), True),
                                        StructField("lap", IntegerType(), True),
                                        StructField("position", IntegerType(), True),
                                        StructField("time", StringType(), True),
                                        StructField("milliseconds", IntegerType(), True)])

In [4]:
lap_time_df = spark.read.csv(r"E:\unused\Udemy\Spark_practice\raw\raw_files\lap_times", schema = lap_time_schema)
lap_time_df.show(2)

+------+--------+---+--------+--------+------------+
|raceId|driverId|lap|position|    time|milliseconds|
+------+--------+---+--------+--------+------------+
|   841|      20|  1|       1|1:38.109|       98109|
|   841|      20|  2|       1|1:33.006|       93006|
+------+--------+---+--------+--------+------------+
only showing top 2 rows



In [5]:
lap_time_df.count()

490904

In [6]:
lap_time_fnl_df = lap_time_df.withColumnRenamed("raceId","race_id").\
                              withColumnRenamed("driverId","driver_id").\
                              withColumn("ingestion_date",current_timestamp())
lap_time_fnl_df.show(2)

+-------+---------+---+--------+--------+------------+--------------------+
|race_id|driver_id|lap|position|    time|milliseconds|      ingestion_date|
+-------+---------+---+--------+--------+------------+--------------------+
|    841|       20|  1|       1|1:38.109|       98109|2024-01-17 12:05:...|
|    841|       20|  2|       1|1:33.006|       93006|2024-01-17 12:05:...|
+-------+---------+---+--------+--------+------------+--------------------+
only showing top 2 rows



In [7]:
lap_time_fnl_df.write.mode('overwrite').format('parquet').saveAsTable('f1_processed.lap_times')